In [1]:
# Import dependencies
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Read in data
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.15121593, 0.17576867, 0.13266166, 0.04907511, 0.01401733,
       0.01221132, 0.01409163, 0.01406518, 0.02471234, 0.01553941,
       0.01047573, 0.01203542, 0.00927222, 0.01597601, 0.0326252 ,
       0.01769497, 0.01328625, 0.01202508, 0.00991825, 0.01044381,
       0.01046605, 0.01568941, 0.01044784, 0.01017696, 0.00979598,
       0.00730881, 0.04108154, 0.00933344, 0.0091328 , 0.02304463,
       0.01627924, 0.00927024, 0.0095696 , 0.01150771, 0.00974022,
       0.01015067, 0.01035599, 0.01067806, 0.00951516, 0.00934415])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.175769
koi_fpflag_nt        0.151216
koi_fpflag_co        0.132662
koi_fpflag_ec        0.049075
koi_model_snr        0.041082
koi_duration_err1    0.032625
koi_time0bk_err1     0.024712
koi_steff_err1       0.023045
koi_duration_err2    0.017695
koi_steff_err2       0.016279
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model
Using a logistic regression


In [10]:
from sklearn.linear_model import LogisticRegression

# Initialize model
model = LogisticRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
# Score the model
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.834827388899485
Testing Data Score: 0.8386727688787186


# Predict

In [12]:
predictions = model.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
1084,CONFIRMED,CANDIDATE
3380,CANDIDATE,CANDIDATE
4904,CANDIDATE,CANDIDATE
6,CONFIRMED,CONFIRMED
1665,FALSE POSITIVE,FALSE POSITIVE


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the GridSearchCV model
param_grid = {'C':np.logspace(-4, 4, 20),
             'penalty':['l1','l2']}

grid = GridSearchCV(model, param_grid, verbose=3)

In [14]:
# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] C=0.0001, penalty=l1 ............................................
[CV] ................ C=0.0001, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ................ C=0.0001, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ................ C=0.0001, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.00026366508987303583, penalty=l1 ............................
[CV]  C=0.00026

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] . C=0.0006951927961775605, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l1 .............................
[CV] . C=0.0006951927961775605, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l2 .............................
[CV] . C=0.0006951927961775605, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l2 .............................
[CV] . C=0.0006951927961775605, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l2 .............................
[CV] . C=0.0006951927961775605, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l1 .............................
[CV] . C=0.0018329807108324356, penalty=l1, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l1 .............................
[CV] . C=0.0018329807108324356, penalty=l1, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l1 .............................
[CV] .

[CV] .... C=11.288378916846883, penalty=l1, score=0.859, total=   1.4s
[CV] C=11.288378916846883, penalty=l1 ................................
[CV] .... C=11.288378916846883, penalty=l1, score=0.866, total=   0.5s
[CV] C=11.288378916846883, penalty=l1 ................................
[CV] .... C=11.288378916846883, penalty=l1, score=0.845, total=   1.5s
[CV] C=11.288378916846883, penalty=l2 ................................
[CV] .... C=11.288378916846883, penalty=l2, score=0.851, total=   0.0s
[CV] C=11.288378916846883, penalty=l2 ................................
[CV] .... C=11.288378916846883, penalty=l2, score=0.856, total=   0.0s
[CV] C=11.288378916846883, penalty=l2 ................................
[CV] .... C=11.288378916846883, penalty=l2, score=0.845, total=   0.0s
[CV] C=29.763514416313132, penalty=l1 ................................
[CV] .... C=29.763514416313132, penalty=l1, score=0.859, total=   1.4s
[CV] C=29.763514416313132, penalty=l1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   32.9s finished


In [15]:
print(best_model.best_params_)

{'C': 11.288378916846883, 'penalty': 'l1'}


In [19]:
# Assign best parameters to variables
best_c = best_model.best_params_['C']
best_penalty = best_model.best_params_['penalty']

# Predict with `best_model`

In [20]:
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
1084,CONFIRMED,CONFIRMED
3380,CANDIDATE,CANDIDATE
4904,CANDIDATE,CANDIDATE
6,CONFIRMED,CONFIRMED
1665,FALSE POSITIVE,FALSE POSITIVE


In [21]:
print('Best Score: ', best_model.best_score_)

Best Score:  0.8567613961472439


# Predict With GridSearch Parameters

In [22]:
# Initialize model with new parameters
model2 = LogisticRegression(penalty=best_penalty, C=best_c)

# Train the model
model2.fit(X_train_scaled, y_train)

LogisticRegression(C=11.288378916846883, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# Make predictions on test data
df2 = pd.DataFrame({'Actual':y_test, 'Predicted':model2.predict(X_test_scaled)})
df2.head()

,Actual,Predicted
1084,CONFIRMED,CONFIRMED
3380,CANDIDATE,CANDIDATE
4904,CANDIDATE,CANDIDATE
6,CONFIRMED,CONFIRMED
1665,FALSE POSITIVE,FALSE POSITIVE


In [24]:
print(model2.score(X_test_scaled, y_test))

0.8575514874141876
